In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests as rq
import numpy as np
import time
import pickle
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import Counter
import cv2
import itertools
import seaborn as sns
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...


# Scrap des images pour un hashtag

In [ ]:
USERNAME = 'sosorezka@gmail.com'
PASSWORD = 'sofia2001'

In [ ]:
import asyncio
import aiohttp

async def fetch_image(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            image_data = await response.read()
            return image_data
        else:
            return None

In [ ]:
async def scrap_by_hashtag(nb_max = 50, hashtag='chat'):
  print("ok")

  options = webdriver.ChromeOptions()
  options.headless = True  # Exécution sans interface graphique

  # Initialiser le navigateur Chrome sans spécifier executable_path
  driver = webdriver.Chrome(options=options)

  # Accéder à la page de connexion de Pixelfed
  login_url = 'https://pixelfed.social/login'
  driver.get(login_url)

  # Remplir le formulaire de connexion
  username = USERNAME
  password =  PASSWORD

  username_field = driver.find_element(By.NAME, 'email')
  password_field = driver.find_element(By.NAME, 'password')

  username_field.send_keys(username)
  password_field.send_keys(password)

    # Soumettre le formulaire de connexion
  login_button = driver.find_element(By.CSS_SELECTOR, "button.btn.btn-primary.btn-block.btn-lg.font-weight-bold")
  ActionChains(driver).move_to_element(login_button).click().perform()

  driver.implicitly_wait(10)

    # Ouvrir une nouvelle fenêtre
  new_window = driver.execute_script("window.open('https://pixelfed.social/i/web/hashtag/{}', 'new_window');".format(hashtag))

  # Changer la fenêtre active
  driver.switch_to.window(driver.window_handles[-1])

  liste_url = []
  liste_img = []

  async with aiohttp.ClientSession() as session:
    tasks = []
    while len(liste_url) < nb_max:
      try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(1)

        post =driver.find_element(By.CSS_SELECTOR, "div.row.mx-0.hashtag-feed")
        # Trouver les images
        images = post.find_elements(By.CSS_SELECTOR, "img")
        for image in images:
          image_url = image.get_attribute('src')
          if image_url not in liste_url:
            tasks.append(fetch_image(session, image_url))
            print(len(tasks))
            liste_url.append(image_url)

      except Exception as e:
        print(f"Erreur: {e}")

    downloaded_images = await asyncio.gather(*tasks)

    for image_data in downloaded_images:
      if image_data is not None:
        img = np.array(Image.open(BytesIO(image_data)))
        if img.shape[2] == 3 and img.shape[0] > 512 and img.shape[1] > 512:
            liste_img.append(img)

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.quit()

    df = pd.DataFrame({'url': liste_url, 'image': liste_img})

    with open(f'dataframe_{hashtag}.pkl', 'wb') as file:
        pickle.dump(df, file)

    return df


In [ ]:
test = scrap_by_hashtag(nb_max=1500, hashtag='bird')

Exception ignored in: <coroutine object scrap_by_hashtag at 0x7f0d27e4e180>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'


# Test de l'enregistrement du pickle

In [ ]:
with open('dataframe_cat.pkl', 'rb') as file :
    df = pickle.load(file)
df.head(2)